In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, r2_score, ndcg_score

In [2]:
animes = pd.read_csv("Anime.csv")
animes

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
0,16498,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",TV,25.0,Finished Airing,"Apr 7, 2013",...,"Gore, Military, Survival",Shounen,24.0,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,3524109,155695
1,1535,Death Note,DN,デスノート,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37.0,Finished Airing,"Oct 4, 2006",...,Psychological,Shounen,23.0,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,3504535,159701
2,5114,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi Fullmetal Alchemist, ...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64.0,Finished Airing,"Apr 5, 2009",...,Military,Shounen,24.0,R - 17+ (violence & profanity),9.131,900398.0,12.0,3,2978455,207772
3,30276,One Punch Man,"One Punch-Man, One-Punch Man, OPM",ワンパンマン,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12.0,Finished Airing,"Oct 5, 2015",...,"Parody, Super Power",Seinen,24.0,R - 17+ (violence & profanity),8.511,19066.0,1112.0,4,2879907,59651
4,11757,Sword Art Online,"S.A.O, SAO",ソードアート・オンライン,Sword Art Online,Ever since the release of the innovative Nerve...,TV,25.0,Finished Airing,"Jul 8, 2012",...,"Love Polygon, Video Game",Unknown,23.0,PG-13 - Teens 13 or older,7.201,990254.0,29562.0,5,2813565,64997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21455,45064,Xiao Gongzhu Ai Wei La Yu Shenmi Wangguo,Unknown,小公主艾薇拉与神秘王国,Princess and the Kingdom,Unknown,Movie,1.0,Finished Airing,"May 12, 2018",...,Unknown,Kids,78.0,G - All Ages,NaN,NaN,160892.0,21487,7,0
21456,45065,Xiao Gongzhu Ai Wei La Yu Shenmi Wangguo 2,Unknown,小公主艾薇拉与神秘王国2,Avera and the Mystical Kingdom 2,Unknown,Movie,1.0,Finished Airing,"Aug 22, 2020",...,Unknown,Kids,80.0,G - All Ages,NaN,NaN,160902.0,21488,6,0
21457,45059,Yongzhe Chuang Mo Cheng,Unknown,勇者闯魔城,Brave City,Unknown,Movie,1.0,Finished Airing,"Mar 16, 2018",...,Unknown,Unknown,80.0,G - All Ages,NaN,NaN,164032.0,21489,6,0
21458,45063,Yu Shen Chuan: Wangzhe Shaonian,Unknown,禹神传之王者少年,The Juvenile of King Yu,Unknown,Movie,1.0,Finished Airing,"Aug 15, 2018",...,Unknown,Unknown,90.0,G - All Ages,NaN,NaN,164462.0,21490,6,0


In [3]:
sub_df = animes[['ID','Japanese','English','Type','Episodes','Status','Duration_Minutes','Rating','Members','Favorites','Popularity']].dropna()
sub_df

,ID,Japanese,English,Type,Episodes,Status,Duration_Minutes,Rating,Members,Favorites,Popularity
0,16498,進撃の巨人,Attack on Titan,TV,25.0,Finished Airing,24.0,R - 17+ (violence & profanity),3524109,155695,1
1,1535,デスノート,Death Note,TV,37.0,Finished Airing,23.0,R - 17+ (violence & profanity),3504535,159701,2
2,5114,鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist Brotherhood,TV,64.0,Finished Airing,24.0,R - 17+ (violence & profanity),2978455,207772,3
3,30276,ワンパンマン,One Punch Man,TV,12.0,Finished Airing,24.0,R - 17+ (violence & profanity),2879907,59651,4
4,11757,ソードアート・オンライン,Sword Art Online,TV,25.0,Finished Airing,23.0,PG-13 - Teens 13 or older,2813565,64997,5
...,...,...,...,...,...,...,...,...,...,...,...
21455,45064,小公主艾薇拉与神秘王国,Princess and the Kingdom,Movie,1.0,Finished Airing,78.0,G - All Ages,7,0,21487
21456,45065,小公主艾薇拉与神秘王国2,Avera and the Mystical Kingdom 2,Movie,1.0,Finished Airing,80.0,G - All Ages,6,0,21488
21457,45059,勇者闯魔城,Brave City,Movie,1.0,Finished Airing,80.0,G - All Ages,6,0,21489
21458,45063,禹神传之王者少年,The Juvenile of King Yu,Movie,1.0,Finished Airing,90.0,G - All Ages,6,0,21490


In [4]:
X = sub_df[['Episodes','Duration_Minutes','Rating']]
X = pd.get_dummies(X, columns = ['Rating'])
X

,Episodes,Duration_Minutes,Rating_G - All Ages,Rating_None,Rating_PG - Children,Rating_PG-13 - Teens 13 or older,Rating_R - 17+ (violence & profanity),Rating_R+ - Mild Nudity,Rating_Rx - Hentai
0,25.0,24.0,0,0,0,0,1,0,0
1,37.0,23.0,0,0,0,0,1,0,0
2,64.0,24.0,0,0,0,0,1,0,0
3,12.0,24.0,0,0,0,0,1,0,0
4,25.0,23.0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
21455,1.0,78.0,1,0,0,0,0,0,0
21456,1.0,80.0,1,0,0,0,0,0,0
21457,1.0,80.0,1,0,0,0,0,0,0
21458,1.0,90.0,1,0,0,0,0,0,0


In [5]:
y = sub_df['Popularity']
y

0            1
1            2
2            3
3            4
4            5
         ...  
21455    21487
21456    21488
21457    21489
21458    21490
21459    21490
Name: Popularity, Length: 20640, dtype: int64

In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
result = pd.DataFrame(columns=["model", "r2"])

In [8]:
# Linear Regression
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["Linear", r2_score(y_val,y_pred)]

In [9]:
# lasso
reg = Lasso(alpha=0.1).fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["Lasso", r2_score(y_val,y_pred)]

In [10]:
# ridge
reg = Ridge(alpha=0.1).fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["Ridge", r2_score(y_val, y_pred)]

In [11]:
# decision tree
reg = DecisionTreeRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["DecisionTree", r2_score(y_val, y_pred)]

In [12]:
# random forest
reg = RandomForestRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["Random Forest", r2_score(y_val, y_pred)]

In [13]:
# GBDT
reg = GradientBoostingRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["Gradient Boosting", r2_score(y_val, y_pred)]

In [14]:
# AdaBoost
reg = AdaBoostRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["AdaBoost", r2_score(y_val, y_pred)]

In [15]:
# XGBoost
reg = xgb.XGBRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["XGBoost", r2_score(y_val, y_pred)]

In [16]:
# MLP
reg = MLPRegressor().fit(X_train, y_train)
y_pred = reg.predict(X_val)
result.loc[len(result)] = ["MLP", r2_score(y_val, y_pred)]

D:\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [17]:
result.sort_values(by=["r2"], ascending=False)

,model,r2
7,XGBoost,0.575374
5,Gradient Boosting,0.559148
4,Random Forest,0.542306
3,DecisionTree,0.504410
6,AdaBoost,0.442794
8,MLP,0.417871
1,Lasso,0.396541
2,Ridge,0.396539
0,Linear,0.396539
